In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.colors as colors
from pathlib import Path

def calculate_ndvi_differences(base_year_path, comparison_years_paths, output_dir):
    # Creating output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Reading the base year (2016) NDVI raster
    with rasterio.open(base_year_path) as src_base:
        ndvi_base = src_base.read(1)
        profile = src_base.profile
    
    # Storing statistics 
    stats = []
    colors_list = ['darkred', 'red', 'black', 'green', 'darkgreen']
    cmap = colors.LinearSegmentedColormap.from_list("custom", colors_list)
    # Processing each comparison year
    for comparison_path in comparison_years_paths:
        # Extracting year from filename (assuming format contains year like NDVI_YYYY.tif)
        year = comparison_path.split('NDVI_')[-1].split('.')[0]
        
        # Reading the comparison year NDVI raster
        with rasterio.open(comparison_path) as src_comp:
            ndvi_comp = src_comp.read(1)
        
        # Calculating the difference (2016 - comparison_year)
        ndvi_diff = ndvi_comp - ndvi_base
        
        # Creating the figure and plot
        fig, ax = plt.subplots(figsize=(15, 10))
        
        # Plot
        im = ax.imshow(ndvi_diff, cmap=cmap, vmin=-0.5, vmax=0.5)
        ax.set_title(f'NDVI Difference (2016 - {year})')
        plt.colorbar(im, ax=ax, label='NDVI Difference')
        
        # Removing axis ticks
        ax.set_xticks([])
        ax.set_yticks([])
        
        # text box with statistics
        stats_text = f'Mean diff: {np.nanmean(ndvi_diff):.3f}\n'
        stats_text += f'Max decrease: {np.nanmin(ndvi_diff):.3f}\n'
        stats_text += f'Max increase: {np.nanmax(ndvi_diff):.3f}'
        
        plt.text(0.02, 0.98, stats_text,
                transform=ax.transAxes,
                fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Saving the figure
        output_path = f'{output_dir}/ndvi_difference_2016_{year}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        # Saving the difference raster
        diff_output_path = f'{output_dir}/ndvi_difference_2016_{year}.tif'
        with rasterio.open(diff_output_path, 'w', **profile) as dst:
            dst.write(ndvi_diff, 1)
        
        # Storing statistics
        stats.append({
            'year': year,
            'mean_diff': np.nanmean(ndvi_diff),
            'max_decrease': np.nanmin(ndvi_diff),
            'max_increase': np.nanmax(ndvi_diff)
        })
        
    return stats


if __name__ == "__main__":
    # Base directory 
    base_dir = "../vegetation_indices/vegetation_indices"
    
    # Base year (2016) 
    base_year_path = f"{base_dir}/NDVI_2016.tif"
    
    # Generating paths for comparison years (2017-2025)
    comparison_years = range(2017, 2026)
    comparison_paths = [f"{base_dir}/NDVI_{year}.tif" for year in comparison_years]
    
    # Output directory 
    output_dir = "../ndvi_change_maps/dry"
    
    # Calculating differences and get statistics
    stats = calculate_ndvi_differences(base_year_path, comparison_paths, output_dir)
    

    print("\nNDVI difference statistics:")
    print("Year\tMean Diff\tMax Decrease\tMax Increase")
    print("-" * 50)
    for stat in stats:
        print(f"{stat['year']}\t{stat['mean_diff']:.3f}\t{stat['max_decrease']:.3f}\t{stat['max_increase']:.3f}")


NDVI difference statistics:
Year	Mean Diff	Max Decrease	Max Increase
--------------------------------------------------
2017	-0.003	-0.693	0.668
2018	-0.134	-0.708	0.571
2019	-0.032	-0.577	0.630
2020	-0.032	-0.610	0.654
2021	0.027	-0.702	0.692
2022	-0.193	-0.803	0.572
2023	-0.020	-0.754	0.677
2024	-0.014	-0.929	0.677
2025	-0.008	-0.846	0.660


#### trend line

In [2]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.colors as colors
from pathlib import Path

def calculate_ndvi_differences(base_year_path, comparison_years_paths, output_dir):
    # Creating output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Reading the base year (2016) NDVI raster
    with rasterio.open(base_year_path) as src_base:
        ndvi_base = src_base.read(1)
        profile = src_base.profile
    
    # Store statistics 
    stats = []
    years = []
    max_decreases = []
    max_increases = []
    
    colors_list = ['darkred', 'red', 'black', 'green', 'darkgreen']
    cmap = colors.LinearSegmentedColormap.from_list("custom", colors_list)
    
    # Processing each comparison year
    for comparison_path in comparison_years_paths:
        # Extracting year from filename
        year = int(comparison_path.split('NDVI_')[-1].split('.')[0])
        years.append(year)
        
        # Reading the comparison year NDVI raster
        with rasterio.open(comparison_path) as src_comp:
            ndvi_comp = src_comp.read(1)
        
        # Calculating the difference
        ndvi_diff = ndvi_comp - ndvi_base
        
        # Store the max decrease and increase values
        max_decrease = np.nanmin(ndvi_diff)
        max_increase = np.nanmax(ndvi_diff)
        max_decreases.append(max_decrease)
        max_increases.append(max_increase)
        
        # Creating the difference map (same as before)
        fig, ax = plt.subplots(figsize=(15, 10))
        im = ax.imshow(ndvi_diff, cmap=cmap, vmin=-0.5, vmax=0.5)
        ax.set_title(f'NDVI Difference (2016 - {year})')
        plt.colorbar(im, ax=ax, label='NDVI Difference')
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Save statistics and maps
        stats_text = f'Mean diff: {np.nanmean(ndvi_diff):.3f}\n'
        stats_text += f'Max decrease: {max_decrease:.3f}\n'
        stats_text += f'Max increase: {max_increase:.3f}'
        
        plt.text(0.02, 0.98, stats_text,
                transform=ax.transAxes,
                fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        output_path = f'{output_dir}/ndvi_difference_2016_{year}.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        # Save the difference raster
        diff_output_path = f'{output_dir}/ndvi_difference_2016_{year}.tif'
        with rasterio.open(diff_output_path, 'w', **profile) as dst:
            dst.write(ndvi_diff, 1)
        
        stats.append({
            'year': year,
            'mean_diff': np.nanmean(ndvi_diff),
            'max_decrease': max_decrease,
            'max_increase': max_increase
        })
    
    # Creating trend line plot
    plt.figure(figsize=(12, 6))
    plt.plot(years, max_decreases, 'r-', label='Max Decrease', marker='o')
    plt.plot(years, max_increases, 'g-', label='Max Increase', marker='o')
    plt.xlabel('Year')
    plt.ylabel('NDVI Change')
    plt.title('NDVI Change Trends (2017-2025)')
    plt.legend()
    plt.grid(True)
    
   
    # Saving trend line plot
    plt.savefig(f'{output_dir}/ndvi_trends.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return stats

if __name__ == "__main__":
    # Base directory 
    base_dir = "../vegetation_indices-20250210T155902Z-001/vegetation_indices"
    
    # Base year (2016) 
    base_year_path = f"{base_dir}/NDVI_2016.tif"
    
    # Generating paths for comparison years (2017-2025)
    comparison_years = range(2017, 2026)
    comparison_paths = [f"{base_dir}/NDVI_{year}.tif" for year in comparison_years]
    

    output_dir = "../ndvi_change_maps"
    
    # Calculating differences and get statistics
    stats = calculate_ndvi_differences(base_year_path, comparison_paths, output_dir)
    

    print("\nNDVI difference statistics:")
    print("Year\tMean Diff\tMax Decrease\tMax Increase")
    print("-" * 50)
    for stat in stats:
        print(f"{stat['year']}\t{stat['mean_diff']:.3f}\t{stat['max_decrease']:.3f}\t{stat['max_increase']:.3f}")


NDVI difference statistics:
Year	Mean Diff	Max Decrease	Max Increase
--------------------------------------------------
2017	-0.019	-0.514	0.479
2018	-0.097	-0.570	0.441
2019	-0.087	-0.590	0.427
2020	-0.057	-0.573	0.482
2021	-0.034	-0.646	0.477
2022	-0.053	-0.804	0.447
2023	-0.049	-0.896	0.476
2024	-0.051	-0.907	0.483
2025	-0.046	-0.881	0.479


#### AREA

In [3]:
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
from pathlib import Path

def reproject_raster(src_path, dst_crs='EPSG:3857'):
    with rasterio.open(src_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        
        dst_array = np.zeros((src.count, height, width), dtype=src.dtypes[0])
        
        reproject(
            source=rasterio.band(src, 1),
            destination=dst_array[0],
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )
        
        return dst_array[0], transform

def calculate_area(tiff_path, threshold=0.7):
    data, transform = reproject_raster(tiff_path)
    
    pixel_width = abs(transform[0])
    pixel_height = abs(transform[4])
    pixel_area = pixel_width * pixel_height
    
    # Calculating total valid area
    total_mask = ~np.isnan(data)
    total_pixels = np.count_nonzero(total_mask)
    total_area_km2 = (total_pixels * pixel_area) / 1000000
    
    # Calculating forest area (NDVI >= 0.7)
    binary_mask = np.where(data >= threshold, 1, 0)
    high_value_pixels = np.count_nonzero(binary_mask)
    forest_area_km2 = (high_value_pixels * pixel_area) / 1000000
    
    return forest_area_km2, total_area_km2

def main(folder_path):
    years = []
    forest_areas = []
    total_areas = []
    
    print("\nCalculating areas...")
    print("-" * 50)
    print(f"{'Year':<10} {'Forest (km²)':<15} {'Total (km²)':<15} {'Forest %':<10}")
    print("-" * 50)
    
    for file in sorted(Path(folder_path).glob('*ndvi*.tif')):
        year = int(file.stem.split('_')[1])
        forest_area, total_area = calculate_area(str(file))
        forest_percentage = (forest_area / total_area) * 100 if total_area > 0 else 0
        
        years.append(year)
        forest_areas.append(forest_area)
        total_areas.append(total_area)
        
        print(f"{year:<10} {forest_area:>8.2f} km²    {total_area:>8.2f} km²    {forest_percentage:>6.1f}%")

    if years:
        # Plot 
        plt.figure(figsize=(10, 6))
        plt.plot(years, forest_areas, 'bo-')
        plt.title('Forest Area by Year (NDVI ≥ 0.7)')
        plt.xlabel('Year')
        plt.ylabel('Area (km²)')
        plt.grid(True)
        plt.xticks(years, rotation=45)
        
        # Adding value labels
        for i, area in enumerate(forest_areas):
            plt.annotate(f'{area:.1f}', 
                        (years[i], area), 
                        textcoords="offset points", 
                        xytext=(0,10), 
                        ha='center')
        
        plt.tight_layout()
        plt.savefig('../forest_areas/forest_area.png')
        plt.close()
        
        print("\nSummary Statistics:")
        print("-" * 50)
        print(f"Average forest area: {np.mean(forest_areas):.2f} km²")
        print(f"Average total area: {np.mean(total_areas):.2f} km²")
        print(f"Average forest percentage: {(np.mean(forest_areas) / np.mean(total_areas)) * 100:.1f}%")
        
    else:
        print("No NDVI files found in the specified folder!")

if __name__ == "__main__":
    folder_path = "../vegetation_indices-20250210T155902Z-001/vegetation_indices"  # Update this path
    main(folder_path)


Calculating areas...
--------------------------------------------------
Year       Forest (km²)    Total (km²)     Forest %  
--------------------------------------------------
2016         229.80 km²      313.99 km²      73.2%
2017         187.18 km²      313.99 km²      59.6%
2018          26.81 km²      313.99 km²       8.5%
2019           5.08 km²      313.99 km²       1.6%
2020          73.50 km²      313.99 km²      23.4%
2021         169.69 km²      313.99 km²      54.0%
2022          95.54 km²      313.99 km²      30.4%
2023         127.99 km²      313.99 km²      40.8%
2024         118.10 km²      313.99 km²      37.6%
2025         144.84 km²      313.99 km²      46.1%

Summary Statistics:
--------------------------------------------------
Average forest area: 117.85 km²
Average total area: 313.99 km²
Average forest percentage: 37.5%


#### Cumulative loss

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path
from rasterio.warp import calculate_default_transform, reproject, Resampling

def reproject_raster(src_path, dst_crs='EPSG:3857'):
    """
    Reproject a raster to a new coordinate reference system
    """
    with rasterio.open(src_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        
        dst_array = np.zeros((src.count, height, width), dtype=src.dtypes[0])
        
        reproject(
            source=rasterio.band(src, 1),
            destination=dst_array[0],
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )
        
        return dst_array[0], transform

def calculate_cumulative_loss(input_folder, ndvi_threshold=0.7, target_crs='EPSG:3857'):
    """
    Calculate cumulative forest loss based on NDVI threshold using numpy.cumsum()
    """
    # Getting sorted list of NDVI files
    ndvi_files = sorted(Path(input_folder).glob('*ndvi*.tif'))
    
    years = []
    forest_areas = []
    
    for file in ndvi_files:
        # Extract year from filename
        year = int(file.stem.split('_')[1])
        
        # Reproject raster
        ndvi, transform = reproject_raster(str(file), target_crs)
        
        # Calculate pixel area in the projected CRS
        pixel_area = abs(transform[0] * transform[4])  # m²
        
        # Calculate forest area
        forest_pixels = np.sum(ndvi >= ndvi_threshold)
        forest_area = forest_pixels * pixel_area / 1000000  # Convert to km²
        
        years.append(year)
        forest_areas.append(forest_area)
    
    # Converting to numpy array
    forest_areas = np.array(forest_areas)
    
    # Calculating loss relative to initial forest area
    initial_forest = forest_areas[0]
    losses = initial_forest - forest_areas  # This ensures losses are positive and increasing
    
    cumulative_loss = losses.cumsum()
    
    return years, cumulative_loss.tolist()

def plot_cumulative_loss(years, loss_values, output_path='../forest_areas/cumulative_loss.png'):
    """
    Create and save cumulative loss plot
    """
    plt.figure(figsize=(10, 6))
    plt.plot(years, loss_values, 'r-o', linewidth=2)
    plt.title('Cumulative Forest Loss (NDVI Threshold: 0.7)')
    plt.xlabel('Year')
    plt.ylabel('Cumulative Loss (km²)')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Add value labels
    for i, loss in enumerate(loss_values):
        plt.annotate(f'{loss:.1f}', 
                    (years[i], loss),
                    textcoords="offset points",
                    xytext=(0, 10),
                    ha='center')
    
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def main():

    input_folder = "../vegetation_indices-20250210T155902Z-001/vegetation_indices"  # Update this path
    
    # Calculating cumulative loss
    years, loss_values = calculate_cumulative_loss(input_folder)
    

    plot_cumulative_loss(years, loss_values)
    

    print(f"Total forest loss: {loss_values[-1]:.2f} km²")
    print(f"Average annual loss: {np.mean(np.diff(loss_values)):.2f} km²/year")

if __name__ == "__main__":
    main()

Total forest loss: 1119.45 km²
Average annual loss: 124.38 km²/year
